In [2]:
import numpy as np
import json
import tensorflow as tf
from tensorflow  import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing  import LabelEncoder

In [3]:
with open('/home/hgidea/Desktop/Coding/Python/hackthon/hack4bioheritage/intents.json') as file:
  data=json.load(file)


In [4]:
training_Sentences=[]
training_labels=[]
labels=[]
responses=[]

In [5]:
# Extract training sentences and labels from intents data
for intent in data['intents']:
  for pattern in intent['patterns']:
    training_Sentences.append(pattern)
    training_labels.append(intent['tag'])
  responses.append(intent['responses'])
  if intent['tag'] not in labels:
    labels.append(intent['tag'])



In [6]:
# Encode labels using LabelEncoder
lbl=LabelEncoder()
lbl.fit(training_labels)
training_labels=lbl.transform(training_labels)

In [7]:
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 0, 6, 6, 6, 5, 5, 5,
       5, 5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [8]:
# Define parameters for tokenization and padding
vocab_size = 1000  # Maximum number of words to keep
embedding_dim = 16
max_len = 20  # Maximum length of sentences

In [9]:
# Tokenize training sentences
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(training_Sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_Sentences)
padded_sequences = pad_sequences(sequences, truncating="post", maxlen=max_len)

In [10]:
# Define the model architecture
num_classes = len(labels)
model=Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))



/home/hgidea/Desktop/Coding/Python/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [11]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [12]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
epochs=250
history=model.fit(padded_sequences,np.array(training_labels),epochs=epochs)

Epoch 1/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9099 - loss: 0.3560 
Epoch 2/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9099 - loss: 0.3563 
Epoch 3/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9099 - loss: 0.3625 
Epoch 4/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9099 - loss: 0.3591 
Epoch 5/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9099 - loss: 0.3653 
Epoch 6/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9099 - loss: 0.3606 
Epoch 7/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9203 - loss: 0.3414 
Epoch 8/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9099 - loss: 0.3573 
Epoch 9/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9203 - loss: 0.3377 
Epoch 10/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9504 - loss: 0.3318 
Epoch 11/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9400 - loss: 0.3477 
Epoch 12/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9400 - los

In [22]:
# to save the trained model
import os
chat_model_folder = "/home/hgidea/Desktop/Coding/Python/hackthon/hack4bioheritage/chat_model"

# Generate a unique filename (e.g., chat_model_2024-05-14.keras)
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d")
model_filename = f"chat_model_{timestamp}.keras"
model_path = os.path.join(chat_model_folder, model_filename)

# Save the model
model.save(model_path)


import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
import colorama
colorama.init()
from colorama import Fore,Style,Back
import random

In [29]:
import requests
import pyttsx3
import json
from json import load
# Define function to fetch data from API
def fetch_data_from_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Define function to read aloud text
def read_aloud(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# Modify your chat function to include API access and text-to-speech
def chat():
    # Load the pre-trained model, tokenizer, and label encoder
    model = keras.models.load_model('/home/hgidea/Desktop/Coding/Python/hackthon/hack4bioheritage/chat_model')
    tokenizer = load('/home/hgidea/Desktop/Coding/Python/hackthon/hack4bioheritage/tokenizer.pickle')
    lbl = load('/home/hgidea/Desktop/Coding/Python/hackthon/hack4bioheritage/label_encoder.pickle')

    max_len = 20

    while True:
        print(Fore.LIGHTBLUE_EX + "User:" + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            print("Thank you for coming")
            break

        # Check if input is a command to access an API
        if inp.lower() == "access api":
            api_response = fetch_data_from_api('your_api_endpoint')
            if api_response:
                print("API response:", api_response)
            else:
                print("Failed to fetch data from API")

        # Otherwise, proceed with model prediction and response generation
        else:
            result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]), truncating='post', maxlen=max_len))
            tag = lbl.inverse_transform([np.argmax(result)])
            for i in data['intents']:
                if i['tag'] == tag:
                    chatbot_response = np.random.choice(i['responses'])
                    print(Fore.GREEN + "Chatbot:" + Style.RESET_ALL, chatbot_response)
                    read_aloud(chatbot_response)

print(Fore.YELLOW + "Welcome" + Style.RESET_ALL)
chat()
    

Welcome


ValueError: File format not supported: filepath=/home/hgidea/Desktop/Coding/Python/hackthon/hack4bioheritage/chat_model. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(/home/hgidea/Desktop/Coding/Python/hackthon/hack4bioheritage/chat_model, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [ ]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('/home/hgidea/Desktop/Coding/Python/hackthon/hack4bioheritage/chat_model/chat_model_2024-05-13.keras')

    # load tokenizer object
    with open('/home/hgidea/Desktop/Coding/Python/hackthon/hack4bioheritage/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('/home/hgidea/Desktop/Coding/Python/hackthon/hack4bioheritage/label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: 

In [32]:
import keras

# Replace with the exact path to your model file
model_path = "/home/hgidea/Desktop/Coding/Python/hackthon/hack4bioheritage/chat_model/chat_model_2024-05-13.keras"

try:
  model = keras.models.load_model(model_path)
  print("Model loaded successfully!")
except ValueError as e:
  print("Error loading model:", e)


Model loaded successfully!
